In [ ]:
pip install bs4  # Installing bs4 package to use BeautifulSoup

In [ ]:
# Importing important libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
# Uploading training data, data to be predicted and sample submission file
train_data = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',  delimiter="\t", quoting=3)
test_data = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip', delimiter = "\t", quoting= 3 )
submission1 = pd.read_csv("../input/word2vec-nlp-tutorial/sampleSubmission.csv")

In [ ]:
# Analyzing the training data
train_data.head()

In [ ]:
# Cheking for any missing values in training data
train_data.isnull().sum().sum()     # Zero missing values

In [ ]:
# Since we have three columns in which the sentiment column is target variable and id is not required for training so we will pre process the training data containing reviews only
# Each review is in the form of a paragraph with HTML tags, punctuations, numbers and stopwords such as is, are am etc so we need to clean them 
# For cleaning below is the function which uses beautifulsoup, regular expression and natural language toolkit to achieve the above 

# creating train_y as target variable in the form of an array
train_y = np.array(train_data["sentiment"])

# To eliminate stop word we need to download its vacab from nltk
nltk.download('stopwords')

def process(review):
   # review without HTML tags
   review = BeautifulSoup(review).get_text()
   # review without punctuation and numbers
   review = re.sub("[^a-zA-Z]",' ',review)
   # converting into lowercase and splitting to eliminate stopwords
   review = review.lower()
   review = review.split()
   # review without stopwords
   swords = set(stopwords.words("english"))                      # conversion into set for fast searching
   review = [w for w in review if w not in swords]               
   # joining of splitted paragraph by spaces and return
   return(" ".join(review))

In [ ]:
# processing the training data with the help of function defined above

train_x = []
for r in range(len(train_data["review"])):
  if (r+1)%1000 == 0:
    print("No of reviews processed =", r+1)
  train_x.append(process(train_data["review"][r]))


In [ ]:
# Now we have our processed and cleaned training set but it is in the form of text but to train we need to convert it into numerical data
# For that we will use bag of words which is based on the frequency that each word occur in a review within training data and we will go for 5000 most common words

# Initializing the countvectorizer which is a sklearn tool for bag of words
vectorizer = CountVectorizer( max_features = 5000 )
# Now we will use fit_transform which fits the model to learn vocabulary for 5000 most common words and then transform the training data into feature vectors
train_x = vectorizer.fit_transform(train_x)
# conversion into array
train_x = train_x.toarray()


In [ ]:
# Final train_x and train_y
train_x.shape, train_y.shape

In [ ]:
# Analyzing data to be predicted
test_data.head()

In [ ]:
# Checking for any missing values
test_data.isnull().sum().sum()   # No missing values

In [ ]:
# processing the data to be predicted with the help of function defined above

test = []
for r in range(len(test_data["review"])):
  if (r+1)%1000 == 0:
    print("No of reviews processed =", r+1)
  test.append(process(test_data["review"][r]))


In [ ]:
# Converting the text data in data to be predicted into bag of words feature vectors
test = vectorizer.transform(test)
# conversion into array
test = test.toarray()

In [ ]:
# Final data to be predicted
test.shape

In [ ]:
# Creating a model, fitting and seeing the performance on training data

model = RandomForestClassifier(n_estimators = 100)
model.fit(train_x, train_y)
train_predict = model.predict(train_x)
AUC = roc_auc_score(train_y, train_predict)

In [ ]:
# printing AUC
AUC   

In [ ]:
# prediction on data to be predicted
submission2 = model.predict(test)

In [ ]:
# Conversion of predicted arry to dataframe and combining with sample submission
submission2 = pd.DataFrame(submission2)
submission = pd.concat([submission1, submission2], axis =1)

In [ ]:
# Removing the sentiment column from sample submission and renaming the predicted column from 0 to "sentiment"
submission.drop(columns=["sentiment"], inplace=True)
submission = submission.rename(columns = {0 :"sentiment"})

In [ ]:
# Final submission file
submission

In [ ]:
# saving the final dubmission file in csv format
submission.to_csv("Submission.csv", index = False)